# Chapter 3: RAG Agents with Azure AI Search

In this chapter, you'll learn how to build **Retrieval-Augmented Generation (RAG)** agents that ground their responses in your enterprise knowledge. We'll start from first principles, building a RAG system manually, and then show how the Agent Framework's **Context Providers** abstract away the complexity.

---

## Learning Objectives

By the end of this chapter, you will:

1. Understand the RAG pattern and why it's essential for enterprise AI
2. Build a "raw" RAG implementation manually using Azure AI Search
3. Refactor to use Context Providers as an abstraction layer
4. Combine multiple knowledge sources with automatic context injection
5. (Optional) Create search indexes programmatically using Python

---

## Prerequisites

Before starting this notebook, complete the setup guide: `03.0-setup-guide.md`

You should have:
- An Azure AI Search service with at least one index created
- The health plan documents indexed and vectorized
- Environment variables configured in your `.env` file

In [ ]:
# Environment Setup & Validation
import sys
sys.path.insert(0, '..')

from workshop_utils import validate_env
validate_env()

---

## Part 1: Understanding RAG

### What is Retrieval-Augmented Generation?

**RAG** is a pattern that enhances LLM responses by retrieving relevant information from external knowledge sources before generating an answer.

```
┌─────────────────────────────────────────────────────────────────┐
│                        RAG Architecture                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   User Query ──────┬──────────────────────────────────────────► │
│                    │                                            │
│                    ▼                                            │
│         ┌─────────────────────┐                                 │
│         │   Azure AI Search   │  ◄── Vectorized Documents       │
│         │   (Retrieval)       │                                 │
│         └─────────┬───────────┘                                 │
│                   │                                             │
│                   │ Retrieved Context                           │
│                   ▼                                             │
│         ┌─────────────────────┐                                 │
│         │   LLM (Generation)  │  ◄── Query + Context            │
│         └─────────┬───────────┘                                 │
│                   │                                             │
│                   ▼                                             │
│            Grounded Response                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Why RAG?

| Problem | How RAG Solves It |
|---------|-------------------|
| **Hallucination** | Grounds responses in retrieved facts |
| **Stale Knowledge** | Retrieves from up-to-date indexes |
| **Context Limits** | Only retrieves relevant chunks, not entire documents |
| **Data Privacy** | Keeps proprietary data in your own indexes |

---

## Part 2: Building RAG from Scratch (The Manual Way)

Before using abstractions, let's understand what's happening under the hood. We'll build a RAG system step-by-step:

1. Connect to Azure AI Search
2. Create a search function
3. Manually inject retrieved context into the prompt
4. Generate a response

This "raw" approach gives you full control and helps you understand the mechanics.

### Step 1: Load Environment Variables

First, let's load our configuration. Make sure your `.env` file contains:

```
AZURE_SEARCH_ENDPOINT=https://<search-service-name>.search.windows.net
INDEX_NAME=<your-index-name>
SEARCH_API_KEY=<your-search-api-key>
```

In [ ]:
import os
import json
from dotenv import load_dotenv

load_dotenv()

# Azure AI Search configuration
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("SEARCH_API_KEY")
index_name = os.getenv("INDEX_NAME")

# Azure OpenAI configuration
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_AI_API_VERSION")

print(f"Search Endpoint: {search_endpoint}")
print(f"Index Name: {index_name}")

### Step 2: Create the Search Client

We'll use the Azure AI Search SDK to connect to our index.

In [ ]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Create the search client
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key)
)

print(f"Connected to index: {index_name}")

### Step 3: Perform a Raw Search

Let's see what the search returns **before** we add any LLM processing. This is the "Retrieval" part of RAG.

In [ ]:
def search_knowledge_base(query: str, top_k: int = 3) -> list[dict]:
    """
    Search the Azure AI Search index and return relevant document chunks.
    
    Args:
        query: The search query
        top_k: Number of results to return
        
    Returns:
        List of document chunks with content and scores
    """
    results = search_client.search(
        search_text=query,
        top=top_k,
        query_type="semantic"  # Uses semantic ranking for better relevance
    )
    
    documents = []
    for doc in results:
        documents.append({
            "content": doc.get("chunk", ""),
            "score": doc.get("@search.score", 0),
            "title": doc.get("title", "Unknown")
        })
    
    return documents

# Test the search
test_query = "What is the deductible for Northwind Health Plus?"
results = search_knowledge_base(test_query)

print(f"Query: {test_query}\n")
print(f"Found {len(results)} relevant chunks:\n")

for i, doc in enumerate(results, 1):
    print(f"--- Result {i} (score: {doc['score']:.2f}) ---")
    print(f"Source: {doc['title']}")
    print(f"Content: {doc['content'][:300]}...\n")

### Step 4: Manual RAG - Injecting Context into the Prompt

Now let's manually construct a prompt that includes the retrieved context. This is the core RAG pattern - **retrieve, then generate**.

In [ ]:
from azure.identity.aio import DefaultAzureCredential, get_bearer_token_provider
from agent_framework.azure import AzureOpenAIChatClient

# Setup Azure OpenAI client
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)

chat_client = AzureOpenAIChatClient(
    api_version=api_version,
    azure_endpoint=project_endpoint,
    azure_ad_token_provider=token_provider,
)


async def manual_rag(user_question: str) -> str:
    """
    Perform RAG manually:
    1. Search for relevant documents
    2. Build a prompt with the context
    3. Generate a response
    """
    # Step 1: Retrieve relevant context
    retrieved_docs = search_knowledge_base(user_question, top_k=3)
    
    # Step 2: Format context for the prompt
    context_text = "\n\n".join([
        f"[Source: {doc['title']}]\n{doc['content']}" 
        for doc in retrieved_docs
    ])
    
    # Step 3: Build the augmented prompt
    system_prompt = f"""You are an HR assistant for Contoso Electronics.
Answer questions based ONLY on the following retrieved context.
If the context doesn't contain the answer, say "I don't have that information."

## Retrieved Context:
{context_text}
"""
    
    # Step 4: Generate response using the agent
    agent = chat_client.as_agent(
        name="manual_rag_agent",
        instructions=system_prompt
    )
    
    response = await agent.run(user_question)
    return response.text


# Test manual RAG
question = "What is the deductible for Northwind Health Plus?"
answer = await manual_rag(question)

print(f"Question: {question}\n")
print(f"Answer: {answer}")

### The Problems with Manual RAG

While this works, there are several issues:

| Problem | Description |
|---------|-------------|
| **Boilerplate Code** | You must write retrieval logic for every agent |
| **No Multi-turn Memory** | Context isn't automatically managed across conversation turns |
| **Token Management** | You must manually handle context window limits |
| **Multiple Sources** | Combining multiple indexes requires custom orchestration |
| **Serialization** | Persisting conversation state requires custom code |

This is where **Context Providers** come in.

---

## Part 3: Using Function Tools (Intermediate Approach)

Before jumping to Context Providers, let's look at an intermediate approach: wrapping the search as a **function tool** that the agent can call.

This gives the agent **agency** - it decides when to search, rather than searching on every query.

In [ ]:
from typing import Annotated
from azure.core.exceptions import HttpResponseError


class AzureAISearchTool:
    """
    A function tool that wraps Azure AI Search for use by an agent.
    
    The agent decides when to call this tool based on the user's question.
    """
    
    def __init__(self, search_client: SearchClient):
        self.search_client = search_client
    
    def search(
        self, 
        query: Annotated[str, "The search query to find relevant documents"],
        top: Annotated[int, "Number of results to return"] = 5
    ) -> str:
        """
        Search the Contoso HR knowledge base for information about health plans,
        policies, and employee benefits.
        
        Use this tool to find:
        - Health plan details (Northwind Health Plus, Northwind Standard)
        - Coverage information, deductibles, and copays
        - Eligibility requirements and enrollment procedures
        - Company HR policies and benefits
        """
        try:
            results = self.search_client.search(
                search_text=query,
                top=top,
                query_type="semantic"
            )
            
            documents = []
            for doc in results:
                documents.append({
                    "chunk": doc.get("chunk", ""),
                    "score": doc.get("@search.score", 0)
                })
            
            return json.dumps(documents, indent=2)
            
        except HttpResponseError as e:
            return json.dumps({"error": str(e)})

In [ ]:
# Create the search tool
search_tool = AzureAISearchTool(search_client=search_client)

# Create an agent with the search tool
tool_agent = chat_client.as_agent(
    name="hr_assistant",
    instructions="""
    You are an HR Assistant for Contoso Electronics specializing in employee health plans.
    
    IMPORTANT RULES:
    1. ALWAYS use the search tool before answering questions about health plans
    2. Base your responses ONLY on information from the search results
    3. If the search returns no relevant information, say so explicitly
    4. Cite which plan (Northwind Health Plus or Standard) the information comes from
    """,
    tools=[search_tool.search]
)

# Test the tool-based agent
response = await tool_agent.run("What are the out-of-network copays?")
print(response.text)

### Tools vs. Context Providers: When to Use Which?

| Approach | Use When |
|----------|----------|
| **Function Tool** | Agent should decide when to search (optional retrieval) |
| **Context Provider** | Every query needs grounding (automatic retrieval) |

For RAG scenarios where you always want to ground the agent's responses, **Context Providers** are the better choice.

---

## Part 4: Context Providers - The Abstraction Layer

**Context Providers** are middleware components in the Agent Framework that run before and after every agent invocation. They solve all the problems we identified with manual RAG:

```
┌────────────────────────────────────────────────────────────────────┐
│                    Context Provider Pipeline                       │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  User Message ──► invoking() ──► Agent ──► invoked() ──► Response │
│                      │                         │                   │
│                      │                         │                   │
│                      ▼                         ▼                   │
│              Inject Context             Extract Memories           │
│              (from search)              (for future use)           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### Key Methods

| Method | When It Runs | Purpose |
|--------|--------------|----------|
| `invoking()` | Before LLM call | Inject retrieved context, instructions, or tools |
| `invoked()` | After LLM call | Extract information, update memory, log interactions |

### AzureAISearchContextProvider

The Agent Framework provides a built-in `AzureAISearchContextProvider` that automatically:

1. Searches your index on every user message
2. Injects relevant documents as context
3. Handles authentication and connection management

Here's how the same RAG pattern looks with Context Providers:

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAISearchContextProvider

# Create the context provider - this replaces all our manual search code!
search_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key),
    mode="semantic",  # Use semantic search for better relevance
    top_k=3,          # Retrieve top 3 most relevant documents
)

# Create the agent with the context provider
async with (
    search_provider,
    ChatAgent(
        chat_client=chat_client,
        name="HRAgent",
        instructions="""
        You are a helpful HR assistant for Contoso Electronics employees.
        
        Answer questions using ONLY the context provided to you.
        If the context doesn't contain the answer, say "I don't have that information."
        Be concise and cite specific plan names when applicable.
        """,
        context_providers=[search_provider],
    ) as agent,
):
    # The context provider automatically searches and injects context
    response = await agent.run("What is the deductible for Northwind Health Plus?")
    print(response.text)

### Compare: Manual RAG vs. Context Provider

| Aspect | Manual RAG | Context Provider |
|--------|------------|------------------|
| **Code Lines** | ~30 lines per agent | ~5 lines configuration |
| **Search Timing** | Manual before each call | Automatic on every invocation |
| **Multi-turn** | Custom thread management | Built-in with `AgentThread` |
| **Multiple Sources** | Custom orchestration | Just add more providers |
| **Serialization** | Custom code | Built-in `serialize()` |
| **Token Management** | Manual | Configurable `top_k` |

---

## Part 5: Multiple Knowledge Sources

A powerful feature of Context Providers is combining multiple knowledge sources. Each provider contributes context independently, and the agent receives a unified view.

For this section, you'll need a second index. Follow Exercise 1 below to create it, or use the optional programmatic indexing in Part 6.

### Exercise 1: Create a Second Knowledge Base

Create a second Azure AI Search index using the documents in `data/index2/` folder. This index contains general HR documents:

- Employee benefits overview
- Company policies and handbook
- Wellness programs
- Role descriptions

**Steps:**

1. Upload documents from `data/index2/` to a new container in your storage account
2. Create a new index in Azure AI Search using the portal wizard
3. Use the same vectorization settings as your first index
4. Add the new index name to your `.env` file as `INDEX2_NAME`

<details>
<summary>Click to see detailed instructions</summary>

**1. Create a new blob container:**
- Go to your Storage Account in Azure Portal
- Navigate to Data Storage > Containers
- Click "+ Container" and name it `hr-general-docs`
- Upload all files from `data/index2/`

**2. Create the search index:**
- Go to your Azure AI Search service
- Click "Import data (new)"
- Select Azure Blob Storage as the data source
- Select the RAG scenario
- Point to your new container
- Use `text-embedding-3-large` for vectorization
- Name the index (e.g., `hr-general-index`)

**3. Update your `.env` file:**
```
INDEX2_NAME=hr-general-index
```

</details>

In [ ]:
# Load the second index name
load_dotenv(override=True)  # Reload to get new env vars
index2_name = os.getenv("INDEX2_NAME")

if not index2_name:
    print("INDEX2_NAME not found in .env - please complete Exercise 1 first")
else:
    print(f"Second index configured: {index2_name}")

In [ ]:
# Create two context providers - one for each knowledge base
health_plans_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index_name,  # Health plans index
    credential=AzureKeyCredential(search_key),
    mode="semantic",
    top_k=3,
)

general_hr_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index2_name,  # General HR index
    credential=AzureKeyCredential(search_key),
    mode="semantic",
    top_k=3,
)

# Test questions that span both knowledge bases
TEST_QUESTIONS = [
    "What's ERISA?",  # General HR knowledge
    "What does Northwind Standard not cover?",  # Health plans
    "Can independent contractor services be covered?",  # Health plans
    "What does the VP of Sales do?",  # General HR (role descriptions)
    "What are out-of-network copays?",  # Health plans
]

async with (
    health_plans_provider,
    general_hr_provider,
    ChatAgent(
        chat_client=chat_client,
        name="ComprehensiveHRAgent",
        instructions="""
        You are a helpful HR assistant for Contoso Electronics employees.

        You have access to two knowledge bases:
        1. **Health Plan Details**: Northwind Health Plus and Standard plan specifics
        2. **General HR Knowledge**: Company policies, benefits overview, role descriptions

        Guidelines:
        - Answer using ONLY the context provided
        - Keep responses to 2-3 sentences
        - Cite specific plan names when discussing health coverage
        - If information isn't available, say so clearly
        """,
        context_providers=[health_plans_provider, general_hr_provider],
    ) as hr_agent,
):
    for question in TEST_QUESTIONS:
        print(f"User: {question}")
        print("Agent: ", end="", flush=True)
        
        async for chunk in hr_agent.run_stream(question):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        
        print("\n")

---

## Part 6: Programmatic Index Creation (Optional)

While the Azure Portal wizard is convenient, you may need to create indexes programmatically for:

- CI/CD pipelines
- Dynamic index creation
- Infrastructure as Code
- Testing environments

The Azure AI Search SDK for Python supports full index lifecycle management.

### Creating an Index Programmatically

Here's how to create a vector search index from scratch using Python:

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
)


def create_vector_index(
    index_client: SearchIndexClient,
    index_name: str,
    vector_dimensions: int = 3072  # text-embedding-3-large dimensions
) -> SearchIndex:
    """
    Create a vector search index programmatically.
    
    Args:
        index_client: The SearchIndexClient
        index_name: Name for the new index
        vector_dimensions: Dimensions of your embedding model
        
    Returns:
        The created SearchIndex
    """
    
    # Define the fields
    fields = [
        # Document key - required
        SearchField(
            name="chunk_id",
            type=SearchFieldDataType.String,
            key=True,
            sortable=True,
            filterable=True,
        ),
        # Parent document reference
        SearchField(
            name="parent_id",
            type=SearchFieldDataType.String,
            filterable=True,
        ),
        # The actual text content
        SearchField(
            name="chunk",
            type=SearchFieldDataType.String,
            searchable=True,
        ),
        # Document title/source
        SearchField(
            name="title",
            type=SearchFieldDataType.String,
            searchable=True,
            filterable=True,
        ),
        # Vector field for semantic search
        SearchField(
            name="text_vector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=vector_dimensions,
            vector_search_profile_name="default-vector-profile",
        ),
    ]
    
    # Configure vector search
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(name="default-hnsw"),
        ],
        profiles=[
            VectorSearchProfile(
                name="default-vector-profile",
                algorithm_configuration_name="default-hnsw",
            ),
        ],
    )
    
    # Configure semantic search
    semantic_config = SemanticConfiguration(
        name="default-semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            content_fields=[SemanticField(field_name="chunk")],
            title_field=SemanticField(field_name="title"),
        ),
    )
    
    semantic_search = SemanticSearch(
        configurations=[semantic_config],
        default_configuration_name="default-semantic-config",
    )
    
    # Create the index
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
        semantic_search=semantic_search,
    )
    
    result = index_client.create_or_update_index(index)
    print(f"Created index: {result.name}")
    return result


# Example usage (uncomment to run)
# index_client = SearchIndexClient(
#     endpoint=search_endpoint,
#     credential=AzureKeyCredential(search_key)
# )
# 
# create_vector_index(index_client, "my-programmatic-index")

### Uploading Documents with Embeddings

After creating the index, you need to upload documents with their vector embeddings:

In [ ]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider


def get_embedding(text: str, embedding_client: AzureOpenAI, model: str) -> list[float]:
    """
    Generate an embedding vector for the given text.
    """
    response = embedding_client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding


def upload_documents_with_embeddings(
    search_client: SearchClient,
    documents: list[dict],
    embedding_client: AzureOpenAI,
    embedding_model: str
):
    """
    Upload documents to the search index with generated embeddings.
    
    Args:
        search_client: The SearchClient for the target index
        documents: List of dicts with 'chunk_id', 'chunk', 'title', 'parent_id'
        embedding_client: AzureOpenAI client for embeddings
        embedding_model: Name of the embedding model deployment
    """
    docs_to_upload = []
    
    for doc in documents:
        # Generate embedding for the chunk
        embedding = get_embedding(doc["chunk"], embedding_client, embedding_model)
        
        docs_to_upload.append({
            "chunk_id": doc["chunk_id"],
            "parent_id": doc.get("parent_id", ""),
            "chunk": doc["chunk"],
            "title": doc.get("title", ""),
            "text_vector": embedding,
        })
    
    # Upload in batches
    result = search_client.upload_documents(documents=docs_to_upload)
    print(f"Uploaded {len(docs_to_upload)} documents")
    return result


# Example: Processing PDF files
def chunk_text(text: str, chunk_size: int = 1000, overlap: int = 200) -> list[str]:
    """
    Split text into overlapping chunks.
    """
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap
    
    return chunks


# Example usage (uncomment to run):
# 
# # Setup embedding client
# from azure.identity import DefaultAzureCredential, get_bearer_token_provider
# 
# sync_credential = DefaultAzureCredential()
# sync_token_provider = get_bearer_token_provider(
#     sync_credential, "https://cognitiveservices.azure.com/.default"
# )
# 
# embedding_client = AzureOpenAI(
#     azure_endpoint=project_endpoint,
#     azure_ad_token_provider=sync_token_provider,
#     api_version=api_version
# )
# 
# # Prepare documents
# sample_documents = [
#     {
#         "chunk_id": "doc1-chunk1",
#         "parent_id": "doc1",
#         "chunk": "This is the first chunk of content...",
#         "title": "Sample Document 1"
#     },
#     # ... more documents
# ]
# 
# # Upload with embeddings
# upload_search_client = SearchClient(
#     endpoint=search_endpoint,
#     index_name="my-programmatic-index",
#     credential=AzureKeyCredential(search_key)
# )
# 
# upload_documents_with_embeddings(
#     upload_search_client,
#     sample_documents,
#     embedding_client,
#     "text-embedding-3-large"
# )

### Programmatic Indexing: Key Points

| Step | What You Need |
|------|---------------|
| **1. Create Index** | `SearchIndexClient` with index schema definition |
| **2. Generate Embeddings** | Azure OpenAI embedding model (e.g., `text-embedding-3-large`) |
| **3. Upload Documents** | `SearchClient.upload_documents()` with vectors |
| **4. Configure Semantic Search** | `SemanticConfiguration` with content/title fields |

> **Note**: For production, consider using Azure AI Search's built-in indexers with skillsets, which handle chunking and embedding automatically. The programmatic approach shown here is useful for custom pipelines.

---

## Summary & Key Concepts

### What You Learned

| Concept | Description |
|---------|-------------|
| **RAG Pattern** | Retrieve relevant documents before generating responses to ground the LLM |
| **Manual RAG** | Building retrieval + generation from scratch for full control |
| **Function Tools** | Agent-controlled search (agent decides when to retrieve) |
| **Context Providers** | Automatic context injection before every invocation |
| **AzureAISearchContextProvider** | Built-in provider for Azure AI Search integration |
| **Multiple Sources** | Combining multiple indexes with multiple providers |

### Key Code Patterns

**1. Creating a Context Provider:**
```python
from agent_framework.azure import AzureAISearchContextProvider

provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(api_key),
    mode="semantic",
    top_k=3,
)
```

**2. Using Context Provider with ChatAgent:**
```python
async with (
    provider,
    ChatAgent(
        chat_client=client,
        instructions="Your instructions here",
        context_providers=[provider],
    ) as agent,
):
    response = await agent.run("Your question")
```

**3. Multiple Knowledge Sources:**
```python
context_providers=[provider1, provider2, provider3]
```

### Architecture Decision Guide

| Scenario | Approach |
|----------|----------|
| Always need grounding | Context Provider (automatic) |
| Sometimes need search | Function Tool (agent-controlled) |
| Multiple knowledge bases | Multiple Context Providers |
| Custom retrieval logic | Implement `ContextProvider` base class |
| CI/CD index creation | Programmatic with `SearchIndexClient` |

### Next Steps

- Explore other context providers: `Mem0Provider` for long-term memory, `RedisProvider` for state management
- Check out [Azure AI Search Agentic Retrieval](https://learn.microsoft.com/en-us/azure/search/agentic-retrieval-overview) for advanced multi-query RAG
- Learn about thread serialization for persisting conversations with context

### Additional Resources

- [Agent Framework Context Provider Samples](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/context_providers)
- [Azure AI Search Documentation](https://learn.microsoft.com/en-us/azure/search/)
- [Agent Framework Memory Tutorial](https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/memory)